In [5]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: C:\Users\Apurva\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV


In [7]:
data = pd.read_csv('merged_file.csv')

In [8]:
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

In [9]:
# Check for remaining NaN values
nan_count = data.isna().sum().sum()
print(f"Total NaN values: {nan_count}")

Total NaN values: 99401


In [10]:
data.fillna(method='bfill', inplace=True)

In [11]:
# Check the number of NaN values
print("Number of NaN values in the dataset:")
print(data.isnull().sum())

Number of NaN values in the dataset:
Heat pump heating water supply temperature             0
Roof Top Unit * supply air temperature setpoint_001    0
Roof Top Unit * supply air temperature setpoint_002    0
Roof Top Unit * supply air temperature setpoint_003    0
Roof Top Unit * supply air temperature setpoint_004    0
                                                      ..
Zone temperature of exterior zone068                   0
Zone temperature of exterior zone069                   0
Zone temperature of exterior zone070                   0
Zone temperature of exterior zone071                   0
Zone temperature of exterior zone072                   0
Length: 114, dtype: int64


In [12]:
X = data.drop(['Miscellaneous electric load for the South Wing', 'Miscellaneous electric load for the North Wing',
               'Lighting load for the South Wing', 'Heating Ventilation and Air Conditioning load for the South Wing',
               'Heating Ventilation and Air Conditioning load for the North Wing'], axis=1)
y = data[['Miscellaneous electric load for the South Wing', 'Miscellaneous electric load for the North Wing',
          'Lighting load for the South Wing', 'Heating Ventilation and Air Conditioning load for the South Wing',
          'Heating Ventilation and Air Conditioning load for the North Wing']]

In [13]:
# Use a fixed split technique (e.g., train on the first 80% of the data, test on the remaining 20%)
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [14]:
# Random Forest model
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

# Calculate RMSE for Random Forest
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))
print("Random Forest RMSE:", rf_rmse)


Random Forest RMSE: 8.51457776196998


In [15]:
param_grid = {
    'n_estimators': [100, 200, 300],  # You can specify a range of values
    'max_depth': [None, 10, 20, 30],
    # Add more hyperparameters to tune
}


In [16]:
grid = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
best_model = grid.best_estimator_


In [ ]:
rf_predictions = best_model.predict(X_test)


In [ ]:
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))
print("Random Forest RMSE:", rf_rmse)

mae = mean_absolute_error(y_test, rf_predictions)
print("Mean Absolute Error (MAE):", mae)

r2 = r2_score(y_test, rf_predictions)
print("R-squared (R^2):", r2)


In [ ]:
print("Best Hyperparameters:", grid.best_params_)

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, rf_predictions)
print("Mean Absolute Error (MAE):", mae)

# Calculate R-squared (R^2)
r2 = r2_score(y_test, rf_predictions)
print("R-squared (R^2):", r2)


Mean Absolute Error (MAE): 6.569529586582666
R-squared (R^2): -4.210345324189758
